# Preliminaries

In [1]:
import pandas as pd  #importing all the important packages
import numpy as np
import matplotlib.pyplot as plt #currently not used
import seaborn as sns #currently not used
import pickle

We reformate the dataframe to make it more useful for analysis including using the pokemon name as the index instead of it's pokedex number. 

In [2]:
#plot style
plt.style.use('fivethirtyeight')
#load file with data
pickle_in = open('pokemon.pickle','rb')
df = pickle.load(pickle_in)
#change into upper case
df.columns = df.columns.str.upper().str.replace('_', '') 
#some values in TYPE2 are so they have to be filled or deleted
df['TYPE2'] = df['TYPE2'].replace(to_replace=[None], value=np.nan, inplace=True) #change None to NaN in Type2
df['TYPE2'].fillna(df['TYPE1'], inplace=True) #fill NaN values in Type2 with corresponding values of Type
#change and set the index to the name attribute
df = df.set_index('NAME')
#drop the columns with axis=1
#axis=0 is for rows
df=df.drop(['ID'],axis=1)

In [3]:
print('The columns of the dataset are: ',df.columns) #show the dataframe columns
print('The shape of the dataframe is: ',df.shape)    #shape of the dataframe

The columns of the dataset are:  Index(['HP', 'ATK', 'DEF', 'SPATK', 'SPDEF', 'SPD', 'TYPE1', 'TYPE2',
       'ABILITY1', 'ABILITY2', 'ABILITY3', 'TOTAL', 'GENERATION', 'LEGENDARY'],
      dtype='object')
The shape of the dataframe is:  (905, 14)


The dataframe has 14 columns and 905 rows accounting for the full pokedex up until generation 8. We now print the dataframe (in pokedex order).

In [4]:
df

,HP,ATK,DEF,SPATK,SPDEF,SPD,TYPE1,TYPE2,ABILITY1,ABILITY2,ABILITY3,TOTAL,GENERATION,LEGENDARY
NAME,,,,,,,,,,,,,,
Bulbasaur,45,49,49,65,65,45,grass,grass,Overgrow,Chlorophyll,None,318,1,False
Ivysaur,60,62,63,80,80,60,grass,grass,Overgrow,Chlorophyll,None,405,1,False
Venusaur,80,82,83,100,100,80,grass,grass,Overgrow,Chlorophyll,None,525,1,False
Charmander,39,52,43,60,50,65,fire,fire,Blaze,Solar Power,None,309,1,False
Charmeleon,58,64,58,80,65,80,fire,fire,Blaze,Solar Power,None,405,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ursaluna,130,140,105,45,80,50,ground,ground,Guts,Bulletproof,Unnerve,550,8,False
Basculegion,120,112,65,80,75,78,water,water,Rattled,Adaptability,Mold Breaker,530,8,False
Sneasler,80,130,60,40,80,120,fighting,fighting,Pressure,Poison Touch,None,510,8,False


# Initial Analysis

In [5]:
#Initial stats of dataframe (rounded to 0 dp)
df_summary = df.describe().round(0)
df_summary

,HP,ATK,DEF,SPATK,SPDEF,SPD,TOTAL,GENERATION
count,905.0,905.0,905.0,905.0,905.0,905.0,905.0,905.0
mean,69.0,77.0,72.0,70.0,70.0,66.0,424.0,4.0
std,26.0,30.0,29.0,29.0,27.0,28.0,112.0,2.0
min,1.0,5.0,5.0,10.0,20.0,5.0,175.0,1.0
25%,50.0,55.0,50.0,46.0,50.0,45.0,320.0,2.0
50%,65.0,75.0,68.0,65.0,65.0,65.0,440.0,4.0
75%,80.0,98.0,90.0,90.0,85.0,85.0,500.0,6.0
max,255.0,181.0,230.0,173.0,230.0,200.0,720.0,8.0


# Strongest Pokemon by Type

We consider all the pokemon in the dataset and ordering by 'TOTAL' we consider which is the strongest of each typing. We do this for both primary typing and secondary typing seperately. The way this is done programmatically is we order all the pokemon based on the value of 'TOTAL' and then keep only the first appearance of each typing. E.g. Arceus is the first normal type and any further normal types are dropped from the dataframe.

In [6]:
type1 = df.drop(['TYPE2'],axis=1) #drop column 'TYPE 2'
type1 = type1.sort_values(by='TOTAL', ascending=False) #descending order by 'TOTAL'
type1 = type1.drop_duplicates(subset=['TYPE1'],keep='first') #since the rows are now sorted in descending order

To display the table we drop unnecessary columns (e.g. 'ABILITY' and 'GENERATION') and we also highlight values the pokemon has which are in the top or bottom 25% of pokemon stats who are fully evolved, shown in green or red respectively (these numbers were calculated externally).

In [7]:
type1.drop(['ABILITY1'],axis=1).drop(['ABILITY2'],axis=1).drop(['ABILITY3'],axis=1).style.apply(lambda x: ["background: green" if((i==0 and v >= 95) or (i==1 and v>=110) or (i==2 and v>=90) or (i==3 and v>99) or (i==4 and v>=90) or (i==5 and v>100)) else "" for i, v in enumerate(x)], axis = 1).apply(lambda x: ["background: red" if((i==0 and v <= 70) or (i==1 and v<=75) or (i==2 and v<=65) or (i==3 and v<=60) or (i==4 and v<63) or (i==5 and v<55)) else "" for i, v in enumerate(x)], axis=1)

,HP,ATK,DEF,SPATK,SPDEF,SPD,TYPE1,TOTAL,GENERATION,LEGENDARY
NAME,,,,,,,,,,
Arceus,120,120,120,120,120,120,normal,720,4,True
Eternatus,140,85,95,145,95,130,poison,690,8,True
Xerneas,126,131,95,131,98,99,fairy,680,6,True
Giratina,150,100,120,100,120,90,ghost,680,4,True
Palkia,90,120,100,150,120,100,water,680,4,True
Dialga,100,120,120,150,100,90,steel,680,4,True
Mewtwo,106,110,90,154,90,130,psychic,680,1,True
Reshiram,100,120,100,150,120,90,dragon,680,5,True
Yveltal,126,131,95,131,98,99,dark,680,6,True


Note these are all legendary pokemon, which we would most likely expect. We now do the same for the secondary typing.

In [8]:
type2 = df.drop(['TYPE1'],axis=1) #drop 'TYPE 1'
type2 = type2.sort_values(by='TOTAL', ascending=False) #descending order by 'TOTAL'
type2 = type2.drop_duplicates(subset=['TYPE2'],keep='first') #since the rows are now sorted in descending order
type2.drop(['ABILITY1'],axis=1).drop(['ABILITY2'],axis=1).drop(['ABILITY3'],axis=1).style.apply(lambda x: ["background: green" if((i==0 and v >= 95) or (i==1 and v>=110) or (i==2 and v>=90) or (i==3 and v>99) or (i==4 and v>=90) or (i==5 and v>100)) else "" for i, v in enumerate(x)], axis = 1).apply(lambda x: ["background: red" if((i==0 and v <= 70) or (i==1 and v<=75) or (i==2 and v<=65) or (i==3 and v<=60) or (i==4 and v<63) or (i==5 and v<55)) else "" for i, v in enumerate(x)], axis=1)

,HP,ATK,DEF,SPATK,SPDEF,SPD,TYPE2,TOTAL,GENERATION,LEGENDARY
NAME,,,,,,,,,,
Arceus,120,120,120,120,120,120,normal,720,4,True
Eternatus,140,85,95,145,95,130,poison,690,8,True
Xerneas,126,131,95,131,98,99,fairy,680,6,True
Giratina,150,100,120,100,120,90,ghost,680,4,True
Palkia,90,120,100,150,120,100,water,680,4,True
Dialga,100,120,120,150,100,90,steel,680,4,True
Mewtwo,106,110,90,154,90,130,psychic,680,1,True
Reshiram,100,120,100,150,120,90,dragon,680,5,True
Yveltal,126,131,95,131,98,99,dark,680,6,True


# STRONGEST POKEMON BY INDIVDUAL STATS

We now sort the pokemon by individual base stats, we can see which pokemon have the highest scores in HP, speed (SPD), attack (ATK), defence (DEF), special attack (SPATK) and special defence (SPDEF). Note again we use the red and green markings to highlight top and bottom quartile rankings of the pokemon's stat. We also drop all the columns we do not need for these rankings ('TYPE', 'ABILITY', 'GENERATION', etc)

In [9]:
#DROP EXTRA COLUMNS (INCLUDING TYPES)
stats=df.drop(['TYPE1'],axis=1).drop(['TYPE2'],axis=1).drop(['ABILITY1'],axis=1).drop(['ABILITY2'],axis=1).drop(['ABILITY3'],axis=1) #drop the columns with axis=1;axis=0 is for rows

In [10]:
# sort pokemon by speed and print the top 10 fastest (.head(10))
spd = stats.sort_values(by='SPD', ascending=False).drop(['GENERATION'],axis=1)
spd.head(10).style.apply(lambda x: ["background: green" if((i==0 and v >= 95) or (i==1 and v>=110) or (i==2 and v>=90) or (i==3 and v>99) or (i==4 and v>=90) or (i==5 and v>100)) else "" for i, v in enumerate(x)], axis = 1).apply(lambda x: ["background: red" if((i==0 and v <= 70) or (i==1 and v<=75) or (i==2 and v<=65) or (i==3 and v<=60) or (i==4 and v<63) or (i==5 and v<55)) else "" for i, v in enumerate(x)], axis=1)

,HP,ATK,DEF,SPATK,SPDEF,SPD,TOTAL,LEGENDARY
NAME,,,,,,,,
Regieleki,80,100,50,100,50,200,580,True
Ninjask,61,90,45,50,50,160,456,False
Pheromosa,71,137,37,137,37,151,570,True
Electrode,60,50,70,80,80,150,490,False
Deoxys,50,150,50,150,50,150,600,True
Accelgor,80,70,40,100,60,145,495,False
Zeraora,88,112,75,102,80,143,600,True
Dragapult,88,120,75,100,75,142,600,False
Zacian,92,130,115,80,115,138,670,True


In [11]:
# sort pokemon by HP and print the top 10 highest (.head(10))
hp = stats.sort_values(by='HP', ascending=False).drop(['GENERATION'],axis=1)
hp.head(10).style.apply(lambda x: ["background: green" if((i==0 and v >= 95) or (i==1 and v>=110) or (i==2 and v>=90) or (i==3 and v>99) or (i==4 and v>=90) or (i==5 and v>100)) else "" for i, v in enumerate(x)], axis = 1).apply(lambda x: ["background: red" if((i==0 and v <= 70) or (i==1 and v<=75) or (i==2 and v<=65) or (i==3 and v<=60) or (i==4 and v<63) or (i==5 and v<55)) else "" for i, v in enumerate(x)], axis=1)

,HP,ATK,DEF,SPATK,SPDEF,SPD,TOTAL,LEGENDARY
NAME,,,,,,,,
Blissey,255,10,10,75,135,55,540,False
Chansey,250,5,5,35,105,50,450,False
Guzzlord,223,101,53,97,53,43,570,True
Regidrago,200,100,50,100,50,80,580,True
Wobbuffet,190,33,58,33,58,33,405,False
Wailord,170,90,45,90,45,60,500,False
Alomomola,165,75,80,40,45,65,470,False
Snorlax,160,110,65,65,110,30,540,False
Giratina,150,100,120,100,120,90,680,True


In [12]:
# sort pokemon by attack and print the top 10 fastest (.head(10))
atk = stats.sort_values(by='ATK', ascending=False).drop(['GENERATION'],axis=1)
atk.head(10).style.apply(lambda x: ["background: green" if((i==0 and v >= 95) or (i==1 and v>=110) or (i==2 and v>=90) or (i==3 and v>99) or (i==4 and v>=90) or (i==5 and v>100)) else "" for i, v in enumerate(x)], axis = 1).apply(lambda x: ["background: red" if((i==0 and v <= 70) or (i==1 and v<=75) or (i==2 and v<=65) or (i==3 and v<=60) or (i==4 and v<63) or (i==5 and v<55)) else "" for i, v in enumerate(x)], axis=1)

,HP,ATK,DEF,SPATK,SPDEF,SPD,TOTAL,LEGENDARY
NAME,,,,,,,,
Kartana,59,181,131,59,31,109,570,True
Rampardos,97,165,60,65,50,58,495,False
Slaking,150,160,100,95,65,100,670,False
Regigigas,110,160,110,80,110,100,670,True
Deoxys,50,150,50,150,50,150,600,True
Rayquaza,105,150,90,150,90,95,680,True
Groudon,100,150,140,100,90,90,670,True
Zekrom,100,150,120,120,100,90,680,True
Haxorus,76,147,90,60,70,97,540,False


In [13]:
# sort pokemon by special attack and print the top 10 fastest (.head(10))
spatk = stats.sort_values(by='SPATK', ascending=False).drop(['GENERATION'],axis=1)
spatk.head(10).style.apply(lambda x: ["background: green" if((i==0 and v >= 95) or (i==1 and v>=110) or (i==2 and v>=90) or (i==3 and v>99) or (i==4 and v>=90) or (i==5 and v>100)) else "" for i, v in enumerate(x)], axis = 1).apply(lambda x: ["background: red" if((i==0 and v <= 70) or (i==1 and v<=75) or (i==2 and v<=65) or (i==3 and v<=60) or (i==4 and v<63) or (i==5 and v<55)) else "" for i, v in enumerate(x)], axis=1)

,HP,ATK,DEF,SPATK,SPDEF,SPD,TOTAL,LEGENDARY
NAME,,,,,,,,
Xurkitree,83,89,71,173,71,83,570,True
Mewtwo,106,110,90,154,90,130,680,True
Blacephalon,53,127,53,151,79,107,570,True
Palkia,90,120,100,150,120,100,680,True
Deoxys,50,150,50,150,50,150,600,True
Rayquaza,105,150,90,150,90,95,680,True
Kyogre,100,100,90,150,140,90,670,True
Dialga,100,120,120,150,100,90,680,True
Hoopa,80,110,60,150,130,70,600,True


In [14]:
# sort pokemon by special defence and print the top 10 fastest (.head(10))
spdef = stats.sort_values(by='SPDEF', ascending=False).drop(['GENERATION'],axis=1)
spdef.head(10).style.apply(lambda x: ["background: green" if((i==0 and v >= 95) or (i==1 and v>=110) or (i==2 and v>=90) or (i==3 and v>99) or (i==4 and v>=90) or (i==5 and v>100)) else "" for i, v in enumerate(x)], axis = 1).apply(lambda x: ["background: red" if((i==0 and v <= 70) or (i==1 and v<=75) or (i==2 and v<=65) or (i==3 and v<=60) or (i==4 and v<63) or (i==5 and v<55)) else "" for i, v in enumerate(x)], axis=1)

,HP,ATK,DEF,SPATK,SPDEF,SPD,TOTAL,LEGENDARY
NAME,,,,,,,,
Shuckle,20,10,230,10,230,5,505,False
Regice,80,50,100,100,200,50,580,True
Ho-Oh,106,130,90,110,154,90,680,True
Lugia,106,90,130,90,154,110,680,True
Florges,78,65,68,112,154,75,552,False
Diancie,50,100,150,100,150,50,600,True
Goodra,90,100,70,110,150,80,600,False
Carbink,50,50,150,50,150,50,500,False
Probopass,60,55,145,75,150,40,525,False


In [15]:
# sort pokemon by defence and print the top 10 fastest (.head(10))
defence = stats.sort_values(by='DEF', ascending=False).drop(['GENERATION'],axis=1)
defence.head(10).style.apply(lambda x: ["background: green" if((i==0 and v >= 95) or (i==1 and v>=110) or (i==2 and v>=90) or (i==3 and v>99) or (i==4 and v>=90) or (i==5 and v>100)) else "" for i, v in enumerate(x)], axis = 1).apply(lambda x: ["background: red" if((i==0 and v <= 70) or (i==1 and v<=75) or (i==2 and v<=65) or (i==3 and v<=60) or (i==4 and v<63) or (i==5 and v<55)) else "" for i, v in enumerate(x)], axis=1)

,HP,ATK,DEF,SPATK,SPDEF,SPD,TOTAL,LEGENDARY
NAME,,,,,,,,
Shuckle,20,10,230,10,230,5,505,False
Stakataka,61,131,211,53,101,13,570,True
Regirock,80,100,200,50,100,50,580,True
Steelix,75,85,200,55,65,30,510,False
Avalugg,95,117,184,44,46,28,514,False
Aggron,70,110,180,60,60,50,530,False
Cloyster,50,95,180,85,45,70,525,False
Bastiodon,60,52,168,47,138,30,495,False
Onix,35,45,160,30,45,70,385,False


# Pokemon with 'TOTAL' over 500 and 525

In [16]:
print(sum(stats['TOTAL'] >= 500), sum(stats['TOTAL'] >= 525)) #number of pokemon with base totals >=500 and >=525

254 165


There are an incredible 254 pokemon with totals greater than 500 and 165 greater than 525. If we look at that in each generation we can see the similar numbers of pokemon in each generation have stats of 500 or more. When we consider the number of pokemon in each generation we realise that in generation 1 about 20% of pokemon have stats >= 500, generation 2 is about 25%, generation 6 is about 31%, and generation 8 is about 40%. Using this trend we can see a known concept of 'power creep', as new pokemon need to be stronger to keep the game interesting and influence the game on arrival rather than be unnecessary compared to original pokemon.

In [17]:
gen1 = stats[stats['GENERATION'] == 1]
print("Generation 1:",sum(gen1['TOTAL'] >= 500), sum(gen1['TOTAL'] >= 525), gen1['TOTAL'].count(), '%:',(sum(gen1['TOTAL'] >= 500)/gen1['TOTAL'].count()*100).round(1))
gen2 = stats[stats['GENERATION'] == 2]
print("Generation 2:",sum(gen2['TOTAL'] >= 500), sum(gen2['TOTAL'] >= 525), gen2['TOTAL'].count(), '%:',(sum(gen2['TOTAL'] >= 500)/gen2['TOTAL'].count()*100).round(1))
gen3 = stats[stats['GENERATION'] == 3]
print("Generation 3:",sum(gen3['TOTAL'] >= 500), sum(gen3['TOTAL'] >= 525), gen3['TOTAL'].count(), '%:',(sum(gen3['TOTAL'] >= 500)/gen3['TOTAL'].count()*100).round(1))
gen4 = stats[stats['GENERATION'] == 4]
print("Generation 4:",sum(gen4['TOTAL'] >= 500), sum(gen4['TOTAL'] >= 525), gen4['TOTAL'].count(), '%:',(sum(gen4['TOTAL'] >= 500)/gen4['TOTAL'].count()*100).round(1))
gen5 = stats[stats['GENERATION'] == 5]
print("Generation 5:",sum(gen5['TOTAL'] >= 500), sum(gen5['TOTAL'] >= 525), gen5['TOTAL'].count(), '%:',(sum(gen5['TOTAL'] >= 500)/gen5['TOTAL'].count()*100).round(1))
gen6 = stats[stats['GENERATION'] == 6]
print("Generation 6:",sum(gen6['TOTAL'] >= 500), sum(gen6['TOTAL'] >= 525), gen6['TOTAL'].count(), '%:',(sum(gen6['TOTAL'] >= 500)/gen6['TOTAL'].count()*100).round(1))
gen7 = stats[stats['GENERATION'] == 7]
print("Generation 7:",sum(gen7['TOTAL'] >= 500), sum(gen7['TOTAL'] >= 525), gen7['TOTAL'].count(), '%:',(sum(gen7['TOTAL'] >= 500)/gen7['TOTAL'].count()*100).round(1))
gen8 = stats[stats['GENERATION'] == 8]
print("Generation 8:",sum(gen8['TOTAL'] >= 500), sum(gen8['TOTAL'] >= 525), gen8['TOTAL'].count(), '%:',(sum(gen8['TOTAL'] >= 500)/gen8['TOTAL'].count()*100).round(1))

Generation 1: 33 18 151 %: 21.9
Generation 2: 25 15 100 %: 25.0
Generation 3: 23 19 135 %: 17.0
Generation 4: 41 31 107 %: 38.3
Generation 5: 37 21 156 %: 23.7
Generation 6: 23 14 72 %: 31.9
Generation 7: 33 28 88 %: 37.5
Generation 8: 39 19 96 %: 40.6


In [20]:
#all pokemon with total of 500 plus
stats[stats['TOTAL'] >= 500].sort_values(by='TOTAL', ascending=False).style.apply(lambda x: ["background: green" if((i==0 and v >= 95) or (i==1 and v>=110) or (i==2 and v>=90) or (i==3 and v>99) or (i==4 and v>=90) or (i==5 and v>100)) else "" for i, v in enumerate(x)], axis = 1).apply(lambda x: ["background: red" if((i==0 and v <= 70) or (i==1 and v<=75) or (i==2 and v<=65) or (i==3 and v<=60) or (i==4 and v<63) or (i==5 and v<55)) else "" for i, v in enumerate(x)], axis=1)

,HP,ATK,DEF,SPATK,SPDEF,SPD,TOTAL,GENERATION,LEGENDARY
NAME,,,,,,,,,
Arceus,120,120,120,120,120,120,720,4,True
Eternatus,140,85,95,145,95,130,690,8,True
Rayquaza,105,150,90,150,90,95,680,3,True
Ho-Oh,106,130,90,110,154,90,680,2,True
Dialga,100,120,120,150,100,90,680,4,True
Mewtwo,106,110,90,154,90,130,680,1,True
Palkia,90,120,100,150,120,100,680,4,True
Giratina,150,100,120,100,120,90,680,4,True
Xerneas,126,131,95,131,98,99,680,6,True


Above is the list of all pokemon with 'TOTAL' >= 500. Again we use red and green to mark the strongest and weakest pokemon of each individual stat. It's quite obvious that the pokemon that dominate the table are the pokemon with the 'LEGENDARY' status equal to true. As these pokemon are rare to get in the game it is not always so helpful to include them. Below we consider the same table with legendaries removed.

In [23]:
#all pokemon with total of 500 plus which are not legendarys
noL = stats[stats['LEGENDARY']==False]
noL[noL['TOTAL'] >= 500].sort_values(by='TOTAL', ascending=False).style.apply(lambda x: ["background: green" if((i==0 and v >= 95) or (i==1 and v>=110) or (i==2 and v>=90) or (i==3 and v>99) or (i==4 and v>=90) or (i==5 and v>100)) else "" for i, v in enumerate(x)], axis = 1).apply(lambda x: ["background: red" if((i==0 and v <= 70) or (i==1 and v<=75) or (i==2 and v<=65) or (i==3 and v<=60) or (i==4 and v<63) or (i==5 and v<55)) else "" for i, v in enumerate(x)], axis=1)

,HP,ATK,DEF,SPATK,SPDEF,SPD,TOTAL,GENERATION,LEGENDARY
NAME,,,,,,,,,
Slaking,150,160,100,95,65,100,670,3,False
Metagross,80,135,130,95,90,70,600,3,False
Dragonite,91,134,95,100,100,80,600,1,False
Goodra,90,100,70,110,150,80,600,6,False
Tyranitar,100,134,110,95,100,61,600,2,False
Kommo-o,75,110,125,100,105,85,600,7,False
Salamence,95,135,80,110,80,100,600,3,False
Hydreigon,92,105,90,125,90,98,600,5,False
Dragapult,88,120,75,100,75,142,600,8,False
